# Exercice - Collaborative filtering

Implement user-based collaborative filtering to find recommendations for new user.

In [ ]:
import scipy.stats
import numpy

The code below reads file "movies.csv" with ratings in the following form (userId, itemId, rating) and processes it. As a result in the variable ratings there is a matrix with users as rows and items as colunms. In cells there are ratings.

In [ ]:
def readRatings(path):
    file=open(path, "r")
    lines = file.read().split("\n")
    return([[int(x) for x in line.split(",")] for line in lines if line != ""])

def processRatings(path):
    ratings = readRatings(path)
    maxUser = max([item[0] for item in ratings])
    maxItem = max([item[1] for item in ratings])
    ratMatrix = numpy.zeros((maxUser, maxItem))
    for rat in ratings:
        ratMatrix[rat[0]-1, rat[1]-1] = rat[2]
    return(ratMatrix)
ratings = processRatings("movies.csv")

**TODO** implement similarity for pair of vectors (user ratings). Use Pearson correlation (scipy.stats.pearsonr).

In [ ]:
def similarity(item1, item2):
    return 0.0

In [ ]:
similarity(ratings[0],ratings[1])

**TODO** implement weighted average. RatingsCol parameter contains a column from ratings matrix (ratings of all users for one movie). Weights parameter is the array of similarities of users to current user (non-zero for k nearest neighbors, zeros for others).

In [ ]:
def weightedMean(ratingsCol, weights):
    return 0.0

weightedMean([1,2,3], [5,2,4])

**TODO** implement user-based collaborative filtering. Use the following steps:
    1. find similarities for all users fo given user (parameter userId). Remember not to take into consideration this user himself. The easiest form is to set value -1 for similarity of user to himself. 
    2. sort similarities descending
    3. find weights vector - similarity for k nearest users, 0 for others
    4. find predicted ratings for all items, which werent already rated by this user
        4.1. call weightedMean method for all columns with zeros for given user and computed in step 2 weights vector)
        4.2. sort predicted values descending
    5. return results in the form of sorted descending list of tuples (itemId, predicted rating)

In [ ]:
k=10 #number of closest users used for recommendation
def findRecommendationsUserBased(userId, raitingsMatrix):
    return []

The following code fragment prints 10 recommended movies for 10 first users. Notice that the user and movie IDs corespond the ones from input file, not the matrix indices. The matrix row/column index = user/movie ID - 1

In [ ]:
usersCount = ratings.shape[0]
for user in range(5):
    recommendations = findRecommendationsUserBased(user, ratings)
    for i in range(10):
        print("User: " + str(user + 1) + ", Item: " + str(recommendations[i][0] + 1) + ", predicted rating: " + str(round(recommendations[i][1], 2)))
    print("")